In [ ]:
pip install -q yahoo_fin

In [4]:
mkdir -p data results log

In [ ]:
pip install  git+https://github.com/ahsank/runml#egg=runml

In [ ]:
import runml.pipeline
from runml import pipeline,findata
from google.colab import drive
#from importlib import reload
# reload(runml.pipeline)

In [ ]:
# Skip some days from consecutive window to lessen test /train overlap
# findata.G_NON_OVERLAP= 0

Run following if like to save results in drive

In [ ]:
drive.mount('/content/drive')
! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print


In [7]:
import numpy as np
import pandas as pd

In [8]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    # Simplify following using apply
    def calculate_alloc(row):
        gain_low = min(row['Gain_l'], 0)
        gain_high = max(row['Gain_h'], 0)
        gain = min(row['Gain'], gain_high)
        average_gain = (gain_low+gain_high + 2*gain)/4
        if (average_gain == 0):
          return 0
        avg_accuracy = (row['Accu']+row['Accu_l']+row['Accu_h'])/3
        alloc =  avg_accuracy/stop_loss - (1-avg_accuracy)*profit_factor/abs(average_gain)
        return round(max(alloc, 0), 2)
    df['Alloc'] = df.apply(calculate_alloc, axis=1)
    return df

pd.options.display.max_columns = None

In [9]:
import os
findata.EPOCHS=200
def run_model(prefix, version, tickers, do_train):
  if (not do_train):
    os.system(f"cp  /content/drive/MyDrive/colab/results/{prefix}-{version}-* ./results/")
  mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(),
                         pipeline.AddMA(200), pipeline.Adj()]))
  df= pipeline.runModelCombinedVola(tickers, f"{prefix}-{version}", mod, do_train)
  # Save datafrme to prefix-version-prediction.csv
  df.to_csv(f"results/{prefix}-{version}-prediction.csv")
  return df



In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax' # Change to minmax
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BASE', 'BILL', 'BMBL', 'BYRN', 'CELH', 'CFLT', 'COHR', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DLO', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ELF', 'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HCP', 'HIMS', 'HOOD', 'HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JAKK', 'JKS', 'KD', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL','MTCH', 'MU',
           'NET', 'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PAR', 'PANW',  'PAYX', 'PD', 'PI', 'PINS', 'PUBM',
           'RBLX', 'RKLB', 'RMBS', 'RUM', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SQ', 'SPOT',
           'SOUN', 'TEAM', 'TDOC', 'TMDX', 'TNDM', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
df = run_model('ipos', '12b', tickers, False)

In [ ]:
addAllocHL(df, 0.10, 1)
df[(df.Gain>0)].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,CurrentDate,PredictionDate,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
10,CFLT,1.30,0.86,370.46,470.52,2025-02-04,2025-03-04,29.03,35.31,0.22,0.77,0.71,436.27,532.68,37.80,0.30,0.62,0.85,550.96,727.41,26.84,-0.08,6.89
88,WOLF,6.84,0.83,503.14,1162.95,2025-02-04,2025-03-04,5.71,9.60,0.68,4.97,0.62,304.42,918.95,9.20,0.61,5.00,0.69,251.51,857.07,7.32,0.28,6.51
84,U,5.80,0.84,307.05,695.55,2025-02-04,2025-03-04,21.62,31.52,0.46,5.44,0.67,220.18,805.39,29.46,0.36,3.61,0.72,180.37,781.02,20.60,-0.05,6.44
30,GFS,1.15,0.93,482.55,622.43,2025-02-04,2025-03-04,40.57,46.35,0.14,0.72,0.72,665.51,708.32,47.75,0.18,0.62,0.78,580.17,779.69,40.22,-0.01,6.41
18,DOCN,1.54,0.93,484.71,571.68,2025-02-04,2025-03-04,40.57,45.58,0.12,0.99,0.68,350.38,473.65,45.91,0.13,0.93,0.83,584.06,751.20,39.99,-0.01,6.06
50,MXL,2.90,0.76,216.78,382.36,2025-02-04,2025-03-04,18.74,24.99,0.33,2.17,0.62,105.74,236.71,28.66,0.53,1.52,0.71,204.14,462.71,16.48,-0.12,5.83
77,TDOC,13.61,0.77,136.17,450.37,2025-02-04,2025-03-04,10.28,13.69,0.33,11.48,0.54,-107.99,233.57,16.21,0.58,6.44,0.74,143.58,514.35,12.41,0.21,5.81
32,GLBE,1.07,0.94,518.74,290.40,2025-02-04,2025-03-04,58.75,67.30,0.15,0.61,0.75,550.34,307.79,66.14,0.13,0.58,0.71,468.81,456.93,55.55,-0.05,5.65
87,UPWK,2.08,0.75,99.25,182.64,2025-02-04,2025-03-04,15.63,20.36,0.30,1.82,0.55,11.14,94.97,22.56,0.44,1.26,0.73,92.95,164.08,16.47,0.05,5.52
45,MBLY,1.07,0.89,211.18,313.15,2025-02-04,2025-03-04,15.99,17.53,0.10,0.65,0.74,210.42,367.34,19.08,0.19,0.60,0.68,160.69,231.16,16.29,0.02,5.34


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'CIBR', 'DAPP', 'DIA', 'DTEC', 'DXYZ',
            'EEM', 'FDN', 'FPX',
            'ICLN', 'IGV', 'IJK', 'IJR', 'IPO',
            'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'JXI', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'URA', 'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
df1 = run_model('etf', '1b', tickers1, True)



In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
49,XME,0.68,0.97,392.56,371.59,60.69,55.83,-0.08,0.44,0.79,350.45,315.67,60.42,-0.00,0.55,0.64,363.95,259.80,53.26,-0.12,9.325000
4,DAPP,0.34,0.96,148.18,182.13,16.87,18.75,0.11,0.26,0.71,158.97,145.53,20.63,0.22,0.25,0.66,130.40,107.76,15.50,-0.08,9.236364
7,DXYZ,2.07,0.93,378.84,74.59,62.91,53.21,-0.15,1.87,0.82,350.60,39.15,62.84,-0.00,1.09,0.55,95.70,-318.62,52.98,-0.16,8.833333
6,DTEC,0.35,0.96,209.30,173.01,47.70,49.59,0.04,0.24,0.71,150.19,153.57,48.12,0.01,0.24,0.64,181.06,135.37,45.94,-0.04,8.600000
20,JETS,0.31,0.96,190.02,154.07,26.25,27.24,0.04,0.16,0.78,145.19,145.35,28.84,0.10,0.21,0.66,102.83,77.59,25.30,-0.04,8.600000
40,XLE,0.98,0.95,605.00,432.48,91.31,87.06,-0.05,0.63,0.83,568.18,413.59,92.96,0.02,0.76,0.68,497.27,264.52,85.88,-0.06,8.500000
41,XLF,0.30,0.96,187.98,158.93,50.85,49.49,-0.03,0.21,0.80,179.19,110.36,49.55,-0.03,0.24,0.63,148.60,80.13,45.98,-0.10,8.266667
48,XLY,1.52,0.95,1148.04,855.93,230.01,220.52,-0.04,1.10,0.73,828.58,645.42,230.46,0.00,1.15,0.66,1017.61,701.53,215.19,-0.06,8.250000
24,MTUM,1.52,0.95,913.42,624.58,223.01,214.26,-0.04,0.80,0.82,805.63,523.72,216.08,-0.03,0.89,0.63,620.82,458.37,203.46,-0.09,8.250000
9,FDN,2.02,0.97,1273.08,791.16,258.50,263.32,0.02,1.37,0.72,993.77,611.73,258.72,0.00,1.41,0.67,1184.25,831.61,245.79,-0.05,8.200000


In [ ]:
df1.to_csv('results/etf_12b_predictions.csv')

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax'
tickers3 = [ 'AIRI', 'AEXAF', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS',  'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CHGG', 'CLOV', 'CRCT', 'CXM',
            'DM', 'EVGO', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MITK', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PLUG', 'PTON',
            'REAL', 'REI', 'RIG', 'RVYL',
            'SABR', 'SFIX', 'SLDP', 'SLE', 'SMWB', 'SST', 'STEM',
            'TDW', 'TDOC', 'TRUP', 'VTEX', 'VSAT', 'YEXT']
df3 = run_model('vols', '12b', tickers3, False)

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3[(df3.Gain>0)].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
6,APPS,3.22,0.89,119.07,389.05,2.50,3.22,0.29,3.22,0.62,5.41,273.15,3.30,0.32,1.17,0.68,37.48,364.03,2.18,-0.13,4.855984
37,LUMN,0.57,0.88,46.05,99.02,4.93,6.69,0.36,0.53,0.68,64.04,87.76,6.40,0.30,0.38,0.70,23.08,29.23,4.99,0.01,4.843137
59,SLDP,0.20,0.86,46.25,79.13,1.40,1.92,0.38,0.19,0.60,8.97,43.09,1.83,0.31,0.10,0.73,31.83,68.33,1.37,-0.02,4.690402
46,OPEN,1.51,0.85,52.11,99.35,1.44,2.06,0.43,1.50,0.64,25.98,73.04,2.20,0.53,0.80,0.76,35.35,100.56,1.66,0.16,4.651163
20,DLO,0.76,0.85,153.81,285.68,13.56,19.03,0.40,1.00,0.70,112.43,268.39,22.45,0.66,0.46,0.64,139.60,290.98,12.41,-0.09,4.625000
62,SST,0.33,0.82,60.72,121.44,0.68,1.15,0.69,0.49,0.50,163.68,210.41,1.10,0.62,0.22,0.68,33.06,101.75,0.79,0.16,4.562660
56,RVYL,6.63,0.76,115.83,375.80,1.23,4.90,2.98,6.13,0.54,139.52,278.82,3.67,1.98,1.85,0.66,62.09,232.28,1.84,0.49,4.390051
63,STEM,1.52,0.76,56.41,132.35,0.60,1.89,2.15,1.75,0.56,15.38,86.05,1.20,1.00,0.88,0.65,10.19,68.06,0.56,-0.06,4.358960
11,BKKT,22.91,0.76,447.14,1272.55,18.48,40.71,1.20,38.74,0.47,220.12,994.89,37.53,1.03,7.76,0.74,299.03,1494.75,15.87,-0.14,4.270588
2,ALSMY,0.27,0.88,23.65,23.93,1.96,2.22,0.13,0.29,0.68,10.33,14.72,2.10,0.07,0.24,0.62,9.55,13.06,1.88,-0.04,4.253394


In [ ]:
# read csv file
pred_other = pd.read_csv('results/pennystock_predictions_12b.csv')
# Join with df3['Ticker] by pred_other['ticker']
pred_other = pred_other.merge(df3[['Ticker', 'Pred', 'Pred_h', 'Pred_l', 'Gain', 'Alloc']], left_on='ticker', right_on='Ticker', how='left')



In [ ]:
pred_other.head()

,ticker,current_price,prediction_date,predicted_min,predicted_max,predicted_close,Ticker,Pred,Pred_h,Pred_l,Gain
0,AIRI,4.44,2025-01-24,4.09,4.84,4.66,AIRI,3.65,5.58,3.17,-0.18
1,AEXAF,0.00,2025-01-24,0.00,0.00,0.00,AEXAF,0.90,0.59,0.12,299.97
2,ALSMY,2.22,2025-01-24,2.14,2.32,2.40,ALSMY,2.06,2.44,2.13,-0.07
3,AMSC,25.06,2025-01-24,18.64,25.48,20.27,AMSC,18.58,30.70,15.83,-0.26
4,AMSSY,3.54,2025-01-24,3.06,5.00,3.79,AMSSY,4.44,4.58,2.05,0.26


In [ ]:
# Get all entries where predicted_close > current_price and Gain > 0

result = pred_other[(pred_other['predicted_close'] > pred_other['current_price']) & (pred_other['Gain'] > 0)]
# And largest based on 'Alloc
result = result.sort_values('Alloc', ascending=False)
result


,ticker,current_price,prediction_date,predicted_min,predicted_max,predicted_close,Ticker,Pred,Pred_h,Pred_l,Gain,Alloc
18,CRCT,5.75,2025-01-24,4.63,6.14,6.13,CRCT,7.62,6.88,5.00,0.32,5.071691
24,GOGO,7.69,2025-01-24,6.78,7.75,7.91,GOGO,8.41,8.68,7.34,0.09,5.032680
4,AMSSY,3.54,2025-01-24,3.06,5.00,3.79,AMSSY,4.44,4.58,2.05,0.26,4.812217
46,OPEN,1.67,2025-01-24,1.24,1.80,1.77,OPEN,2.20,2.24,1.55,0.32,4.801471
58,SAVE,0.40,2025-01-24,0.42,0.59,0.56,SAVE,1.42,1.29,0.07,2.56,4.753217
51,PLUG,2.38,2025-01-24,2.25,3.05,2.65,PLUG,4.34,2.93,1.99,0.82,4.746055
44,OMQS,0.17,2025-01-24,0.15,0.26,0.24,OMQS,0.39,0.44,0.15,1.24,4.745256
25,GRAB,4.81,2025-01-24,4.59,5.02,4.92,GRAB,5.44,7.41,3.95,0.13,4.660633
47,OTLY,0.64,2025-01-24,0.57,0.66,0.66,OTLY,1.15,1.04,0.51,0.80,4.455882
64,STEM,0.60,2025-01-24,0.51,0.65,0.62,STEM,1.75,1.03,0.50,1.94,4.410855


In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DELL', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'KO', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']
df4 = run_model('mcap', '1b', tickers4, True)

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
26,META,8.02,0.97,3794.63,1664.59,688.37,610.90,-0.11,6.00,0.78,3836.22,1381.18,688.48,0.00,5.13,0.70,2773.50,735.38,498.78,-0.28,6.655844
4,AMZN,3.10,0.96,1671.51,883.28,233.77,205.71,-0.12,1.74,0.82,1425.03,913.93,234.16,0.00,1.49,0.71,1335.32,1086.15,190.53,-0.18,6.523810
1,ACN,4.67,0.93,2125.91,1720.03,384.83,303.90,-0.21,3.26,0.79,2012.36,1404.43,378.51,-0.02,2.75,0.72,1896.34,1541.58,336.63,-0.13,6.309524
12,COST,11.90,0.91,5137.56,2227.70,978.47,727.57,-0.26,6.99,0.80,4488.53,1642.54,954.29,-0.02,5.77,0.69,4083.76,683.85,827.48,-0.15,6.153846
20,HD,5.05,0.91,2270.81,1404.48,416.38,334.69,-0.20,3.09,0.79,2464.20,1616.96,414.30,-0.01,2.80,0.60,1221.54,429.30,352.79,-0.15,6.050000
18,GOOGL,2.28,0.90,921.36,641.00,199.74,153.12,-0.23,1.80,0.77,1101.88,629.13,198.59,-0.01,1.37,0.62,724.18,480.96,171.00,-0.14,5.993789
19,FDX,4.28,0.92,1677.91,1512.85,265.64,231.55,-0.13,2.98,0.77,2105.18,1793.72,278.14,0.05,2.41,0.72,1902.69,1304.46,229.04,-0.14,5.956044
40,UPS,2.70,0.90,698.42,981.62,111.34,131.96,0.19,1.74,0.76,674.44,1193.59,130.24,0.17,1.58,0.70,668.95,1152.04,121.71,0.09,5.902256
7,BKNG,57.27,0.91,28747.38,14212.10,4758.55,4074.35,-0.14,38.87,0.80,27620.39,14234.91,4895.22,0.03,35.58,0.67,19939.31,7458.83,4108.20,-0.14,5.857143
22,IBM,2.67,0.90,1099.68,625.37,259.31,215.64,-0.17,1.85,0.74,1169.17,445.25,240.17,-0.07,1.56,0.66,914.26,123.03,198.11,-0.24,5.840336


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
tickers5 = ['ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CX', 'DBOEY', 'DNNGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FDRVF', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']

df5 = run_model('gstock', '2a', tickers5, True)

In [ ]:
addAlloc(df5, 0.14)
df5[df5.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,CurrentDate,PredictionDate,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
57,STLA,0.34,0.92,158.26,149.38,2025-02-04,2025-03-04,13.11,15.15,0.16,0.23,0.76,123.34,96.68,13.68,0.04,0.24,0.66,70.24,101.53,11.89,-0.09,5.571429
39,LVMUY,2.33,0.94,1250.78,1065.23,2025-02-04,2025-03-04,143.99,159.07,0.10,1.79,0.65,777.04,797.01,165.09,0.15,1.84,0.70,605.71,755.15,139.19,-0.03,5.514286
63,TRYIY,0.18,0.94,73.30,51.30,2025-02-03,2025-03-03,13.56,14.25,0.05,0.12,0.77,81.49,58.52,14.28,0.05,0.12,0.71,74.96,61.71,12.84,-0.05,4.314286
3,BAK,0.41,0.91,99.60,148.56,2025-02-04,2025-03-04,4.68,5.04,0.08,0.26,0.66,73.25,114.72,6.78,0.45,0.30,0.75,34.64,152.71,4.24,-0.09,4.250000
13,DNNGY,0.89,0.82,149.78,319.69,2025-02-04,2025-03-04,12.48,15.00,0.20,0.66,0.79,155.17,281.36,15.74,0.26,0.73,0.78,113.59,275.90,12.56,0.01,4.057143
4,BASFY,0.19,0.97,94.25,105.73,2025-02-04,2025-03-04,11.66,11.92,0.02,0.11,0.78,81.28,79.45,12.54,0.08,0.12,0.74,67.94,96.64,11.00,-0.06,3.928571
22,EVKIY,0.19,0.89,51.80,84.05,2025-02-03,2025-03-03,8.96,9.67,0.08,0.10,0.78,69.36,72.79,9.53,0.06,0.11,0.70,32.96,71.62,8.89,-0.01,3.607143
35,MGA,1.07,0.91,317.80,504.30,2025-02-04,2025-03-04,37.87,39.98,0.06,0.79,0.76,285.39,417.37,46.01,0.22,0.82,0.67,222.13,370.35,34.68,-0.08,3.500000
61,THQQF,1.59,0.83,253.73,396.29,2025-02-03,2025-03-03,18.30,19.97,0.09,1.21,0.65,147.39,312.13,22.10,0.21,1.09,0.58,90.47,317.38,16.25,-0.11,2.150794
17,ENGGY,0.10,0.89,29.28,41.38,2025-02-03,2025-03-03,6.26,6.55,0.05,0.06,0.80,33.43,46.92,6.40,0.02,0.06,0.66,28.39,35.80,6.00,-0.04,1.957143


In [ ]:
findata.G_SCALER = 'standard'
tickers6 = ['ARGT', 'EWD', 'FLN',
           'ECH', 'EPHE', 'EWA', 'EWG', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWT', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'KWT', 'THD', 'TUR', 'VNM', 'XCEM']
df6 = run_model('etf', '12a', tickers6, False)

In [ ]:
addAllocHL(df6, 0.07, 1)
df6.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,CurrentDate,PredictionDate,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
11,EWP,1.10,0.81,141.60,62.80,2025-02-05,2025-03-05,33.88,27.93,-0.18,0.81,0.70,115.32,57.24,32.03,-0.05,0.85,0.57,54.97,-5.45,27.91,-0.18,7.63
13,EWW,2.76,0.74,158.65,143.62,2025-02-05,2025-03-05,49.53,43.71,-0.12,1.85,0.65,171.98,149.18,49.89,0.01,1.99,0.67,80.52,141.13,43.31,-0.13,6.33
7,EWI,1.52,0.75,124.94,71.43,2025-02-05,2025-03-05,38.92,32.82,-0.16,1.09,0.69,112.33,26.46,37.78,-0.03,0.95,0.53,35.22,-23.00,34.86,-0.10,6.11
22,TUR,1.78,0.75,216.51,110.74,2025-02-05,2025-03-05,35.10,28.75,-0.18,1.21,0.63,181.14,68.51,36.77,0.05,1.27,0.50,71.27,-49.54,28.72,-0.18,5.90
0,ARGT,3.71,0.61,328.57,2.69,2025-02-05,2025-03-05,86.39,69.20,-0.20,2.84,0.64,359.45,-87.61,75.48,-0.13,2.39,0.51,216.35,-98.20,65.19,-0.25,5.84
1,EWD,1.65,0.79,176.28,116.31,2025-02-05,2025-03-05,41.84,36.76,-0.12,1.21,0.61,90.50,73.12,40.03,-0.04,1.18,0.62,81.05,70.48,39.02,-0.07,5.40
8,EWJ,1.99,0.73,201.53,123.89,2025-02-05,2025-03-05,68.84,62.93,-0.09,1.46,0.68,118.48,74.39,68.45,-0.01,1.59,0.60,130.93,91.40,61.41,-0.11,5.02
20,KWT,0.47,0.89,111.70,110.69,2025-02-05,2025-03-05,34.70,32.11,-0.07,0.40,0.63,66.76,51.13,34.76,0.00,0.34,0.66,45.69,65.09,33.05,-0.05,4.63
18,INDA,2.12,0.69,109.06,76.04,2025-02-05,2025-03-05,51.28,45.49,-0.11,1.66,0.57,9.18,-1.12,51.12,-0.00,1.65,0.49,43.38,-9.61,39.83,-0.22,4.55
6,EWG,0.98,0.78,111.54,101.39,2025-02-05,2025-03-05,34.67,32.01,-0.08,0.80,0.60,74.26,16.07,34.15,-0.01,0.83,0.59,44.70,20.30,31.76,-0.08,3.66


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'standard'
tickers7 = ['AAL', 'ADI', 'ALB', 'AMR','ANF', 'APO', 'AQN', 'ARCH', 'ARE', 'ASC',
            'BAH', 'BAX', 'BKR', 'BLK', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IFF', 'IP', 'IPI', 'IVZ',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW', 'ZTS']
df7 = run_model('val', '1a', tickers7, False)

In [12]:
addAllocHL(df7, 0.10, 1)
df7[df7.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,CurrentDate,PredictionDate,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
96,WHR,2.08,0.95,823.72,1056.42,2025-02-12,2025-03-12,98.60,115.64,0.17,1.17,0.76,1021.94,1198.14,118.42,0.20,1.37,0.81,905.34,1398.34,101.92,0.03,7.21
0,AAL,0.44,0.93,153.64,135.54,2025-02-12,2025-03-12,16.44,19.26,0.17,0.29,0.75,155.67,157.07,19.31,0.17,0.27,0.81,177.83,175.70,16.86,0.03,6.97
6,AQN,0.14,0.90,34.86,79.33,2025-02-12,2025-03-12,4.60,5.23,0.14,0.11,0.72,18.76,64.98,6.26,0.36,0.09,0.80,50.94,98.44,4.83,0.05,6.86
13,BGFV,0.30,0.85,43.22,124.77,2025-02-12,2025-03-12,1.44,1.79,0.24,0.23,0.60,36.17,110.71,1.35,-0.07,0.18,0.77,49.91,127.86,-0.68,-1.47,6.69
68,PETS,0.52,0.86,51.19,158.77,2025-02-12,2025-03-12,4.28,6.63,0.55,0.34,0.54,-32.50,116.45,10.32,1.41,0.29,0.73,59.72,158.26,3.58,-0.16,6.61
11,BAX,0.85,0.93,181.11,489.70,2025-02-12,2025-03-12,30.23,32.63,0.08,0.60,0.67,72.92,384.97,39.63,0.31,0.56,0.85,163.04,514.58,29.41,-0.03,6.50
37,F,0.21,0.94,85.38,122.45,2025-02-12,2025-03-12,9.15,10.02,0.09,0.12,0.77,109.67,123.52,11.00,0.20,0.11,0.77,106.90,104.93,9.06,-0.01,6.39
74,RH,8.21,0.92,3598.13,3413.63,2025-02-12,2025-03-12,369.52,407.31,0.10,5.43,0.69,3225.73,3989.27,441.08,0.19,5.79,0.79,3605.49,3624.02,385.49,0.04,5.95
54,KHC,0.34,0.95,155.47,242.23,2025-02-12,2025-03-12,28.50,31.57,0.11,0.22,0.73,135.71,186.18,31.66,0.11,0.22,0.77,161.77,190.18,28.67,0.01,5.94
41,FLO,0.22,0.93,79.01,100.95,2025-02-12,2025-03-12,18.50,19.96,0.08,0.11,0.68,83.57,105.96,22.42,0.21,0.13,0.79,104.40,157.01,19.30,0.04,5.84


In [ ]:
findata.EPOCHS=200
tickers8 = [ 'DBC', 'FXB', 'FXE', 'FXF', 'FXY',
            'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'USO', 'UUP', 'VTIP', 'WEAT']

df8 =  run_model('com', '12a', tickers8, True)

In [14]:
addAllocHL(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,CurrentDate,PredictionDate,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
12,USO,1.56,0.86,427.11,423.33,2025-02-12,2025-03-12,76.65,103.83,0.35,0.94,0.74,525.83,533.82,92.46,0.21,0.85,0.74,601.86,536.13,70.75,-0.08,6.20
15,WEAT,0.12,0.92,39.08,56.30,2025-02-12,2025-03-12,5.08,4.37,-0.14,0.07,0.68,50.93,77.26,5.00,-0.02,0.06,0.83,34.58,71.11,4.66,-0.08,5.99
8,SLV,0.34,0.92,180.11,122.18,2025-02-12,2025-03-12,29.41,26.06,-0.11,0.22,0.79,195.30,151.32,29.26,-0.01,0.20,0.71,151.85,117.42,26.31,-0.11,5.72
5,GLD,1.98,0.88,833.88,408.84,2025-02-12,2025-03-12,268.06,253.92,-0.05,1.13,0.80,1056.80,409.72,261.95,-0.02,1.05,0.64,615.76,148.86,217.45,-0.19,4.61
7,PDBC,0.18,0.89,56.92,57.24,2025-02-12,2025-03-12,13.60,13.27,-0.02,0.10,0.79,67.96,66.11,13.29,-0.02,0.11,0.78,66.61,67.40,12.90,-0.05,0.20
3,FXF,0.38,0.93,183.91,207.75,2025-02-12,2025-03-12,97.38,94.52,-0.03,0.22,0.67,158.60,156.89,97.85,0.00,0.23,0.70,111.44,104.69,95.39,-0.02,0.00
0,DBC,0.26,0.86,66.96,80.78,2025-02-12,2025-03-12,22.38,22.29,-0.00,0.17,0.74,79.04,87.30,22.86,0.02,0.18,0.73,83.42,78.47,21.83,-0.02,0.00
1,FXB,0.54,0.92,238.09,258.73,2025-02-12,2025-03-12,119.85,117.70,-0.02,0.31,0.72,125.82,193.67,122.01,0.02,0.32,0.69,131.26,177.70,118.95,-0.01,0.00
6,ISHG,0.34,0.91,121.41,148.16,2025-02-12,2025-03-12,67.76,65.21,-0.04,0.23,0.67,62.26,113.73,67.57,-0.00,0.24,0.73,79.38,132.66,66.09,-0.02,0.00
4,FXY,0.67,0.89,134.89,228.16,2025-02-12,2025-03-12,59.76,60.85,0.02,0.43,0.63,58.88,164.87,60.61,0.01,0.39,0.74,49.97,212.73,59.26,-0.01,0.00


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
tickers9 = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'BCH-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'NEAR-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']
df9 = run_model('cry', '11b', tickers9, True)

In [ ]:
addAlloc(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
16,XRP-USD,0.04,0.88,22.84,11.20,3.17,1.54,-0.51,0.04,0.59,18.56,5.68,2.52,-0.21,0.03,0.63,7.98,-4.12,1.28,-0.60,6.050420
2,BTC-USD,1472.88,0.89,828623.43,477821.07,105022.42,80147.25,-0.24,1642.52,0.73,686009.31,373635.46,103679.77,-0.01,1181.56,0.56,293891.33,-119210.61,66526.41,-0.37,5.898810
6,ETH-USD,86.08,0.90,47103.70,43865.06,3308.15,3683.19,0.11,74.21,0.69,38953.14,38099.46,3634.56,0.10,63.62,0.70,22605.10,19374.57,2503.14,-0.24,5.519481
11,MATIC-USD,0.06,0.88,13.76,23.47,0.45,0.52,0.15,0.05,0.60,11.83,19.43,0.53,0.16,0.04,0.65,7.39,12.75,0.44,-0.02,5.485714
1,AVAX-USD,2.20,0.84,647.80,904.72,36.49,45.70,0.25,2.26,0.71,812.03,994.39,45.46,0.25,1.48,0.66,381.01,794.93,31.53,-0.14,5.360000
13,SOL-USD,5.24,0.84,2211.50,1783.16,244.88,184.01,-0.25,6.13,0.54,2464.67,1340.92,211.00,-0.14,4.67,0.54,983.97,-61.48,120.30,-0.51,5.360000
15,TRX-USD,0.00,0.82,1.64,0.85,0.24,0.17,-0.30,0.00,0.78,1.80,0.70,0.24,-0.01,0.00,0.44,0.94,-0.70,0.13,-0.44,5.257143
10,LTC-USD,4.77,0.82,1032.44,1390.54,121.24,156.55,0.29,5.36,0.63,910.94,825.72,142.23,0.17,3.77,0.67,852.95,1010.42,102.75,-0.15,5.236453
3,BCH-USD,19.65,0.83,6289.79,6073.08,443.90,521.74,0.18,20.13,0.55,4830.27,3664.11,543.68,0.22,11.68,0.69,3938.39,3714.63,342.29,-0.23,4.984127
14,TON-USD,0.27,0.70,32.78,50.28,0.14,0.81,4.92,0.36,0.65,73.91,100.50,0.70,4.12,0.21,0.62,15.73,32.38,0.36,1.65,4.939024


In [ ]:
def show_prediction(name, suffix):
  df = pd.read_csv(f'results/{name}_{suffix}_predictions.csv')
  df['predicted_return'] = ((df['predicted_max']+df['predicted_min']+df['predicted_close'])/3 - df['current_price'])/df['current_price']
  return df.sort_values('predicted_return', ascending=False).head(50)


In [ ]:

df = show_prediction('etf', '0202')
df

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
38,XLB,88.79,2025-01-31,2025-02-28,88.13,101.84,94.51,0.067988
10,FPX,131.42,2025-01-31,2025-02-28,130.68,147.53,139.43,0.059301
47,XLV,146.87,2025-01-31,2025-02-28,144.84,159.36,158.29,0.049658
13,IPO,46.92,2025-01-31,2025-02-28,46.14,50.67,50.26,0.044828
4,DAPP,15.18,2025-01-31,2025-02-28,14.62,17.00,15.92,0.043917
31,URA,28.31,2025-01-31,2025-02-28,27.68,31.26,29.70,0.043683
32,VNQ,90.55,2025-01-31,2025-02-28,89.14,95.40,97.48,0.038174
11,ICLN,11.37,2025-01-31,2025-02-28,11.16,12.33,11.92,0.038112
30,TDIV,80.75,2025-01-31,2025-02-28,80.02,88.33,82.90,0.037152
28,SMOG,99.15,2025-01-31,2025-02-28,96.63,106.53,105.05,0.036174


In [ ]:
# read csv file
pred_other1 = pd.read_csv('results/global_predictions_1a.csv')
# Join with df3['Ticker] by pred_other['ticker']
pred_other1 = pred_other1.merge(df5[['Ticker', 'Pred', 'Pred_h', 'Pred_l', 'Gain', 'Alloc']], left_on='ticker', right_on='Ticker', how='left')


In [ ]:

result1 = pred_other1[(pred_other1['predicted_close'] > pred_other1['current_price']) & (pred_other1['Gain'] > 0)]
# And largest based on 'Alloc
result1 = result1.sort_values('Alloc', ascending=False)
result1

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,Ticker,Pred,Pred_h,Pred_l,Gain,Alloc
61,THQQF,2.76,2025-01-03,2025-01-31,2.65,2.82,2.78,THQQF,3.45,4.28,2.93,0.25,1.388571
0,ASML,714.36,2025-01-03,2025-01-31,691.26,789.68,757.36,ASML,726.59,790.66,663.80,0.02,0.000000
2,ADYEY,14.78,2025-01-03,2025-01-31,14.16,15.47,15.05,ADYEY,15.40,16.87,14.27,0.04,0.000000
1,ADDDF,239.00,2025-01-03,2025-01-31,229.15,253.52,244.28,ADDDF,251.01,263.10,235.07,0.05,0.000000
3,BAK,3.78,2025-01-03,2025-01-31,3.66,4.16,3.99,BAK,4.45,5.14,4.00,0.18,0.000000
4,BASFY,10.66,2025-01-03,2025-01-31,10.30,11.54,11.00,BASFY,11.01,11.67,10.43,0.03,0.000000
8,COTY,6.81,2025-01-03,2025-01-31,6.58,7.51,7.21,COTY,7.25,8.06,6.70,0.06,0.000000
5,BESIY,137.69,2025-01-03,2025-01-31,132.71,148.89,143.05,BESIY,142.03,151.01,132.25,0.03,0.000000
13,DNNGY,15.97,2025-01-03,2025-01-31,15.25,16.98,16.03,DNNGY,17.36,19.44,15.36,0.09,0.000000
16,EDNMF,36.35,2025-01-03,2025-01-31,33.96,36.22,37.41,EDNMF,37.23,38.80,34.70,0.02,0.000000


In [ ]:
# List stocks sorted by predicted_close_return descendent
df = pd.read_csv('stock_0205_predictions.csv')
df['predicted_return'] = ((df['predicted_max']+df['predicted_min']+df['predicted_close'])/3 - df['current_price'])/df['current_price']
df.sort_values('predicted_return', ascending=False).head(20)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
176,ASPN,11.38,2025-02-05,2025-03-05,10.71,13.10,15.82,0.160808
198,RBRK,75.71,2025-02-05,2025-03-05,74.08,88.74,97.39,0.145643
119,AMR,183.29,2025-02-05,2025-03-05,178.92,217.38,223.76,0.127648
21,CREX,2.44,2025-02-05,2025-03-05,2.30,2.81,3.10,0.121585
141,INDI,4.30,2025-02-05,2025-03-05,4.17,4.70,5.29,0.097674
3,SE,123.46,2025-02-05,2025-03-05,122.13,138.16,145.96,0.096846
245,SQ,86.17,2025-02-04,2025-03-04,84.28,106.29,91.89,0.092646
232,VIV,8.85,2025-02-05,2025-03-05,8.65,10.06,10.25,0.090772
63,SEDG,14.29,2025-02-05,2025-03-05,12.92,17.83,15.95,0.089340
103,LFMD,6.45,2025-02-05,2025-03-05,5.79,9.18,6.08,0.087855


In [15]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print

sending incremental file list
com-12a-adjclose-sh-1-sc-standard-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
com-12a-high-sh-1-sc-standard-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
com-12a-low-sh-1-sc-standard-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras

sent 28,844,336 bytes  received 73 bytes  57,688,818.00 bytes/sec
total size is 57,674,795  speedup is 2.00
